In [1]:
import os
import pandas as pd
import datetime
import json
from datetime import timedelta

In [2]:
from couchbase.exceptions import CouchbaseException
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions

In [3]:
# Configuración de Couchbase
endpoint = "couchbases://cb.rwijg3tkoqodgahr.cloud.couchbase.com"
username = "BDD_POLITICA"
password = "Admin_1234"  # Reemplazar con la contraseña real
bucket_name = "BDD_POLITICA"
scope_name = "OpinionPublica"
collection_name = "IntencionVoto2025"

In [6]:
# Couchbase connection setup
auth = PasswordAuthenticator(username, password)
options = ClusterOptions(auth)
options.apply_profile("wan_development")

cluster = Cluster(endpoint, options)
cluster.wait_until_ready(timedelta(seconds=5))
cb = cluster.bucket(bucket_name)
cb_coll = cb.scope(scope_name).collection(collection_name)

In [7]:
# Cargar el CSV existente
df_votos = pd.read_csv("intencion_voto.csv")

# Contar votos por candidato
df_conteo = df_votos["candidato"].value_counts().reset_index()
df_conteo.columns = ["candidato", "numero_votantes"]

# Unir la información de número de votantes al DataFrame original
df_votos = df_votos.merge(df_conteo, on="candidato", how="left")

# Guardar el CSV actualizado
df_votos.to_csv("intencion_voto_actualizado.csv", index=False, encoding="utf-8-sig")

print("Archivo CSV actualizado con la columna 'numero_votantes'.")

Archivo CSV actualizado con la columna 'numero_votantes'.


In [10]:
# Subir el archivo CSV a Couchbase 
try:
    with open('intencion_voto_actualizado.csv', 'r', encoding='utf-8') as f:
        csv_content = f.read()
    
    cb_coll.insert('cedatos_intencion_votos_2025_csv', {'content': csv_content})
    print("CSV content inserted into Couchbase.")
except CouchbaseException as e:
    print(f"Error inserting CSV content into Couchbase: {e}")
except Exception as e:
    print(f"General error: {e}")

CSV content inserted into Couchbase.
